# RAG - Sistema de asistencia en investigación académica

In [1]:
import pandas as pd
from pinecone import Pinecone
from pinecone import ServerlessSpec
import os
import time
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import json
import pandas as pd
from langdetect import detect
from openai import OpenAI

/Users/nicolasmautone/Github-Projects/NLP_Datasets/rag-investigacion-academica/rag_investigacion_env/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


## Load Corpus - ACL Anthology

In [4]:
df = pd.read_parquet('../acl-publication-info.74k.parquet')

In [5]:
# Get the first 10 rows
df_reduced = df.head(10)
df_reduced

,acl_id,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby,url,publisher,address,year,...,booktitle,author,title,pages,doi,number,volume,journal,editor,isbn
0,O02-2002,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0,14,https://aclanthology.org/O02-2002,None,None,2002,...,International Journal of Computational Linguis...,"Chen, Keh-Jiann and\nYou, Jia-Ming",A Study on Word Similarity using Context Vecto...,37--58,None,None,None,None,None,None
1,L02-1310,,,8220988,8d5e31610bc82c2abc86bc20ceba684c97e66024,93,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Mihalcea, Rada F.",Bootstrapping Large Sense Tagged Corpora,None,None,None,None,None,None,None
2,R13-1042,Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb,10,https://aclanthology.org/R13-1042,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,Proceedings of the International Conference Re...,"Jamison, Emily and\nGurevych, Iryna","Headerless, Quoteless, but not Hopeless? Using...",327--335,None,None,None,None,None,None
3,W05-0819,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55,15,https://aclanthology.org/W05-0819,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,Proceedings of the {ACL} Workshop on Building ...,"Aswani, Niraj and\nGaizauskas, Robert",Aligning Words in {E}nglish-{H}indi Parallel C...,115--118,None,None,None,None,None,None
4,L02-1309,,,18078432,011e943b64a78dadc3440674419821ee080f0de3,12,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Suyaga, Fumiaki and\nTakezawa, Toshiyuki and...",Proposal of a very-large-corpus acquisition me...,None,None,None,None,None,None,None
5,R13-1044,The paper 1 presents a rule-based approach to ...,The paper 1 presents a rule-based approach to ...,2491460,c0f1047fe0f95c367184d494e78bb07b11ee3608,2,https://aclanthology.org/R13-1044,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,Proceedings of the International Conference Re...,"K{\k{e}}dzia, Pawe{\l} and\nMaziarz, Marek",Recognizing semantic relations within {P}olish...,342--349,None,None,None,None,None,None
6,W05-0818,"In this paper we describe LIHLA, a lexical ali...","In this paper we describe LIHLA, a lexical ali...",15322146,ff3f05120d24e5dac2879f25402993bc6355f780,5,https://aclanthology.org/W05-0818,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,Proceedings of the {ACL} Workshop on Building ...,"Caseli, Helena M. and\nNunes, Maria G. V. an...",{LIHLA}: Shared Task System Description,111--114,None,None,None,None,None,None
7,L02-1313,,,649937,c5c1643517ee6646c47b4ee2b8443d4f62ee1ae5,4,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Baldwin, Timothy and\nBilac, Slaven and\nOku...",Enhanced {J}apanese Electronic Dictionary Look-up,None,None,None,None,None,None,None
8,R13-1045,We describe an approach to building a morpholo...,We describe an approach to building a morpholo...,690455,0b125557ba23075532380e88fb990933838975b7,2,https://aclanthology.org/R13-1045,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,Proceedings of the International Conference Re...,"Khaliq, Bilal and\nCarroll, John",Unsupervised Induction of {A}rabic Root and Pa...,350--356,None,None,None,None,None,None
9,W05-0821,Statistical machine translation systems use a ...,Statistical machine translation syst

In [6]:
# print title, abstract and full text of the first row
print(df.iloc[0]['title'])
print(df.iloc[0]['abstract'])
print(df.iloc[0]['full_text'])

A Study on Word Similarity using Context Vector Models
There is a need to measure word similarity when processing natural languages, especially when using generalization, classification, or example -based approaches. Usually, measures of similarity between two words are defined according to the distance between their semantic classes in a semantic taxonomy . The taxonomy approaches are more or less semantic -based that do not consider syntactic similarit ies. However, in real applications, both semantic and syntactic similarities are required and weighted differently. Word similarity based on context vectors is a mixture of syntactic and semantic similarit ies. In this paper, we propose using only syntactic related co-occurrences as context vectors and adopt information theoretic models to solve the problems of data sparseness and characteristic precision. The probabilistic distribution of co-occurrence context features is derived by parsing the contextual environment of each word , an

## Connect to Vector DB

In [9]:
# Create a serverless index
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "academic-papers"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name, 
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    )

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)
pinecone_index = pc.Index(index_name)

In [6]:
#method to transform row to pinecone format
def row_to_pinecone(row):
    return {
        "id": row['acl_id'],
        "values": json.loads(row['embedding']),
        "metadata": {
            "abstract": row['abstract'],
            "title": row['title'],
            "acl_id": row['acl_id'],
            "corpus_paper_id": row['corpus_paper_id'],
            "bd": "acl"
        }
    }

In [10]:
print(pinecone_index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 63523}},
 'total_vector_count': 63523}


## Query the index

### Example of querying the index

In [11]:
query = "I am doing a research about active learning in NLP. Can you help me find some papers about it?" 

### Embedding query

In [12]:
x = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

### Query similar records in the index

In [13]:
results = pinecone_index.query(
    vector=x[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

In [14]:
results.matches[0].metadata['abstract']

'As supervised machine learning methods for addressing tasks in natural language processing (NLP) prove increasingly viable, the focus of attention is naturally shifted towards the creation of training data. The manual annotation of corpora is a tedious and time consuming process. To obtain high-quality annotated data constitutes a bottleneck in machine learning for NLP today. Active learning is one way of easing the burden of annotation. This paper presents a first probe into the NLP research community concerning the nature of the annotation projects undertaken in general, and the use of active learning as annotation support in particular.'

## Load LLM

In [21]:
client = OpenAI(api_key=os.environ['OPEN_AI_KEY'])

## Setting up a prompt

In [22]:
template2 = """
You are an assistant that provides help to a researcher that is looking for academic papers about a specific topic explained in a query.

You have to base your answers on a context that contains the title and abstract of the 3 most relevant papers found to this query. 

Your tasks are:

-Analyze the abstracts and find out if they are really relevant to the research topic included in the query.
-Obtain the similarities and differences between the papers and the topic. Be as concise as possible and go straight to the point.

If you think that any of the papers are not relevant to the query, reply "I didn't find any relevant information to this topic".

If any of the papers are relevant to the query, reply with the following template:

'I have found relevant information about your research topic in the following papers:
    (For each relevant paper)
    - Title: [title]
    - Similarities: [Briefly list key similarities with the topic]
    - Differences: [Briefly list key differences or gaps]
'

Context:

    Title 1: {title1}
    Abstract 1: {abstract1}

    Title 2: {title2}
    Abstract 2: {abstract2}

    Title 3: {title3}
    Abstract 3: {abstract3}

Query: {query}

"""

In [23]:
query_with_context = template2.format(title1=results.matches[0].metadata['title'], 
                                      abstract1=results.matches[0].metadata['abstract'],
                                      title2=results.matches[1].metadata['title'],  
                                      abstract2=results.matches[1].metadata['abstract'],
                                      title3=results.matches[2].metadata['title'], 
                                      abstract3=results.matches[2].metadata['abstract'], 
                                      query=query)

In [24]:
query_with_context

'\nYou are an assistant that provides help to a researcher that is looking for academic papers about a specific topic explained in a query.\n\nYou have to base your answers on a context that contains the title and abstract of the 3 most relevant papers found to this query. \n\nYour tasks are:\n\n-Analyze the abstracts and find out if they are really relevant to the research topic included in the query.\n-Obtain the similarities and differences between the papers and the topic. Be as concise as possible and go straight to the point.\n\nIf you think that any of the papers are not relevant to the query, reply "I didn\'t find any relevant information to this topic".\n\nIf any of the papers are relevant to the query, reply with the following template:\n\n\'I have found relevant information about your research topic in the following papers:\n    (For each relevant paper)\n    - Title: [title]\n    - Similarities: [Briefly list key similarities with the topic]\n    - Differences: [Briefly lis

In [25]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ 
        { "role": "system", "content": "You are a helpful assistant." },
        {
            "role": "user",
            "content": query_with_context,
        },
    ]
)

In [20]:
print(completion.choices[0].message.content)

I have found relevant information about your research topic in the following papers:
- Title: Deep Reinforcement Learning for {NLP}
  - Similarities: This paper directly addresses the application of deep reinforcement learning (DRL) to various NLP tasks, discussing practical solutions and recent advances in the field.
  - Differences: While it covers foundational concepts and various applications, it may lack specific case studies detailing the performance metrics or comparison with traditional NLP methods.

- Title: Tackling Error Propagation through Reinforcement Learning: A Case of Greedy Dependency Parsing
  - Similarities: This paper applies reinforcement learning to a specific NLP task (dependency parsing), focusing on how it mitigates error propagation, which is a significant concern in NLP processes.
  - Differences: It is narrower in scope, focusing specifically on greedy dependency parsing rather than a broader range of NLP applications or methods.

- Title: Offline Reinforce